<a href="https://colab.research.google.com/github/camillerose/Auditory_Stimulus_Generator/blob/master/keras_monkey_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import the necessary packages
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras import applications
from keras.layers.core import Dense
from keras.optimizers import SGD
from keras.models import load_model
from sklearn.datasets import load_files   
from keras.utils import np_utils
from glob import glob
from keras import applications
from keras.preprocessing.image import ImageDataGenerator 
from keras import optimizers
from keras.models import Sequential,Model,load_model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D,GlobalAveragePooling2D
from keras.callbacks import TensorBoard,ReduceLROnPlateau,ModelCheckpoint
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
import pickle
import cv2
import os

Using TensorFlow backend.


In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"

Mounted at /content/drive


In [3]:
path = '/content/drive/My Drive/data_science/projects/monkey_classification/data/new_world_monkeys'
os.listdir(path)

['capuchins',
 'marmosets',
 'night',
 'saki',
 'spider',
 'tamarins',
 'squirrel',
 'woolly',
 'howlers',
 'models']

In [0]:
# loop through each folder and get the input images plus the labels

data = []
labels = []

for folder in os.listdir(path):
  
  folder_path = path + '/' + folder
  os.chdir(folder_path)
  for file in os.listdir(folder_path):

    image = cv2.imread(file) # read in image
    image = cv2.resize(image, (224, 224)) # resize image
    data.append(image) # add image data
    labels.append(folder) # add image label

In [0]:
# scale the raw pixel intensities to the range [0, 1]
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

random.seed(42)
# split data into train and test
(X_train, X_test, Y_train, Y_test) = train_test_split(data,
	labels, test_size=0.25, random_state=42)

In [0]:
# convert the labels from integers to vectors (for 2-class, binary
# classification you should use Keras' to_categorical function
# instead as the scikit-learn's LabelBinarizer will not return a
# vector)
lb = LabelBinarizer()
Y_train = lb.fit_transform(Y_train)
Y_test = lb.transform(Y_test)

In [7]:
print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))

number of training examples = 274
number of test examples = 92
X_train shape: (274, 224, 224, 3)
Y_train shape: (274, 9)
X_test shape: (92, 224, 224, 3)
Y_test shape: (92, 9)


In [8]:
img_height,img_width = 224,224 
num_classes = 9
base_model = applications.resnet50.ResNet50(weights = 'imagenet', include_top = False, 
                                            input_shape = (img_height,img_width,3))

Instructions for updating:
Colocations handled automatically by placer.


/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [9]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.7)(x)
predictions = Dense(num_classes, activation= 'softmax')(x)
model = Model(inputs = base_model.input, outputs = predictions)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
from keras.optimizers import SGD, Adam

sgd = SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
#adam = Adam(lr=0.003)
model.compile(optimizer= sgd, loss='categorical_crossentropy', metrics=['accuracy'])

In [15]:
model.fit(X_train, Y_train, epochs = 10, validation_data=(X_test, Y_test), batch_size = 64)

Train on 274 samples, validate on 92 samples
Epoch 1/10
274/274 [==============================] - 10s 38ms/step - loss: 2.0818 - acc: 0.3212 - val_loss: 1.7306 - val_acc: 0.4239
Epoch 2/10
274/274 [==============================] - 4s 14ms/step - loss: 1.7189 - acc: 0.3905 - val_loss: 1.5209 - val_acc: 0.5217
Epoch 3/10
274/274 [==============================] - 4s 14ms/step - loss: 1.3585 - acc: 0.5219 - val_loss: 1.2429 - val_acc: 0.6196
Epoch 4/10
274/274 [==============================] - 4s 14ms/step - loss: 0.9015 - acc: 0.7007 - val_loss: 1.0113 - val_acc: 0.7065
Epoch 5/10
274/274 [==============================] - 4s 14ms/step - loss: 0.6455 - acc: 0.8029 - val_loss: 0.8187 - val_acc: 0.7500
Epoch 6/10
274/274 [==============================] - 4s 14ms/step - loss: 0.4484 - acc: 0.8540 - val_loss: 0.7015 - val_acc: 0.7717
Epoch 7/10
274/274 [==============================] - 4s 14ms/step - loss: 0.2962 - acc: 0.9343 - val_loss: 0.6239 - val_acc: 0.7935
Epoch 8/10
274/274 [===

In [0]:
preds = model.evaluate(X_test, Y_test)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

92/92 [==============================] - 1s 15ms/step
Loss = 0.605316020872282
Test Accuracy = 0.7500000077745189


In [0]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation